### 1 投影转换

In [1]:
from osgeo import ogr
from osgeo import osr


In [2]:
path_kunming_districts = 'data/Section-2/kunming_districts.shp'


epsg: 4326   
utm: 326+zone -> 32648


In [6]:
path_kunming_districts_utm = 'data/Section-4/kunming_districts_utm.shp'
utm_epsg = 32648
in_ds = ogr.Open(path_kunming_districts)
in_layer = in_ds.GetLayer(0)
## 创建输出矢量数据
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(path_kunming_districts_utm)
out_layer = out_ds.CreateLayer('kunming_districts_utm', geom_type=ogr.wkbPolygon)
for i in range(in_layer.GetFeature(0).GetFieldCount()):
    field_defn = in_layer.GetFeature(0).GetFieldDefnRef(i)
    out_layer.CreateField(field_defn)

### 空间参考
spatialRef = in_layer.GetSpatialRef() 
utm_spatialRef = osr.SpatialReference()
utm_spatialRef.ImportFromEPSG(utm_epsg)
## 定义投影转换
coordinate_transform = osr.CoordinateTransformation(spatialRef, utm_spatialRef)

### 获得要素定义
FeaDefn = in_layer.GetLayerDefn()
##遍历要素
for i in range(in_layer.GetFeatureCount()):
    in_fea = in_layer.GetFeature(i)
    in_geo = in_fea.geometry()
    in_geo.Transform(coordinate_transform)
    out_geo = in_geo
    out_fea = ogr.Feature(FeaDefn)
    out_fea.SetGeometry(out_geo)
    for j in range(FeaDefn.GetFieldCount()):
        field_value = in_fea.GetField(j)
        out_fea.SetField(j, field_value)
    out_layer.CreateFeature(out_fea)

ds = None
out_ds = None

### 保存投影文件
prj_path = path_kunming_districts_utm.replace('.shp', '.prj')
fn = open(prj_path, 'w')
fn.write(utm_spatialRef.ExportToWkt())
fn.close()




2. 添加/删除面积


In [10]:
ds = ogr.Open(path_kunming_districts_utm, 1)
layer = ds.GetLayer(0)
field_defn = ogr.FieldDefn('area(km2)', ogr.OFTReal)
layer.CreateField(field_defn)
ds = None



In [11]:
ds = ogr.Open(path_kunming_districts_utm, 1)
layer = ds.GetLayer(0)
for i in range(layer.GetFeatureCount()):
    fea = layer.GetFeature(i)
    fea_area = fea.geometry().Area()
    fea_area = fea_area/(1000*1000)
    print('area is: ', fea_area)
    fea.SetField('area(km2)', fea_area)
    layer.SetFeature(fea)

ds = None


area is:  377.9959786954036
area is:  864.2952967369314
area is:  638.33361729476
area is:  897.3967316211449
area is:  1866.1175875335907
area is:  499.3639638876685
area is:  1338.5644973111487
area is:  994.6751193008305
area is:  1914.0921754168523
area is:  1681.3428560105956
area is:  828.4756858595108
area is:  4227.345074248596
area is:  3598.342900037223
area is:  1301.0062596518314


3. 获取要素几何中心点，新建中心点图层

In [12]:
path_kunming_districts_centroid = 'data/Section-4/kunming_districts_centroid.shp'
in_ds = ogr.Open(path_kunming_districts, 0)
in_layer = in_ds.GetLayer(0)
in_SpatialRef = in_layer.GetSpatialRef()

### 创建矢量数据
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(path_kunming_districts_centroid)
out_layer = out_ds.CreateLayer('kunming_districts_centroid', geom_type=ogr.wkbPoint, srs=in_SpatialRef)

field_defn = ogr.FieldDefn('fid', ogr.OFTInteger)
out_layer.CreateField(field_defn)
out_fea_defn = out_layer.GetLayerDefn()

for i in range(in_layer.GetFeatureCount()):
    in_fea = in_layer.GetFeature(i)
    out_geo = in_fea.geometry().Centroid()
    out_fea = ogr.Feature(out_fea_defn)
    out_fea.SetGeometry(out_geo)
    out_fea.SetField('fid', i+1)
    out_layer.CreateFeature(out_fea)

out_ds = None
ds = None
